In [200]:
import cv2
# from google.colab.patches import cv2_imshow
import numpy as np

In [201]:
# img_width=480
# img_height=640
img_height=800

In [202]:
image=cv2.imread('images/1.jpg')

In [203]:
# image height and width
height,width=image.shape[:2]
# aspect ratio of image
aspect_ratio=width/height
img_width=int(img_height*aspect_ratio)
# resize image
image=cv2.resize(image,(int(img_height*aspect_ratio),img_height))

In [204]:
def show_image(image):
  # cv2_imshow(image)
  cv2.imshow('image',image)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [205]:
# image=cv2.resize(image,(img_width,img_height))
show_image(image)

In [206]:
image_gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [207]:
show_image(image_gray)

In [208]:
image_blur=cv2.GaussianBlur(image_gray,(5,5),1)

In [209]:
show_image(image_blur)

In [210]:
image_canny=cv2.Canny(image_blur,130,255)
kernel=np.ones((3,3))

In [211]:
image_closed=cv2.erode(cv2.dilate(image_canny,kernel,iterations=3),kernel, iterations=1)


In [212]:
show_image(image_closed)

In [213]:
# finding all contours
image_contours=image.copy()
image_f_contours=image.copy()
contours,hierarchy=cv2.findContours(image_closed,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cv2.drawContours(image_contours,contours,-1,(0,255,0),10)

array([[[ 18,  65, 107],
        [ 16,  58, 102],
        [ 75, 119, 163],
        ...,
        [ 48, 117, 147],
        [ 57, 127, 157],
        [ 53, 118, 151]],

       [[ 43,  85, 131],
        [ 62, 101, 148],
        [  2,  36,  83],
        ...,
        [ 74, 144, 173],
        [ 89, 161, 189],
        [ 61, 134, 164]],

       [[ 48,  88, 138],
        [113, 152, 203],
        [ 98, 137, 186],
        ...,
        [ 19,  82, 112],
        [ 28,  91, 121],
        [ 21,  86, 119]],

       ...,

       [[  6,  54,  96],
        [  3,  49,  91],
        [ 33,  78, 121],
        ...,
        [ 11,  52,  97],
        [ 22,  63, 108],
        [ 31,  72, 115]],

       [[  9,  52,  92],
        [ 16,  64, 104],
        [ 18,  66, 106],
        ...,
        [ 10,  52,  97],
        [ 43,  84, 129],
        [  2,  39,  83]],

       [[ 25,  78, 116],
        [ 27,  79, 116],
        [ 20,  72, 109],
        ...,
        [ 47,  90, 133],
        [ 68, 112, 155],
        [ 64, 111, 152]]

In [214]:
show_image(image_contours)

In [215]:
# biggest contour
def find_biggest_contour(contours):
  contours=sorted(contours,key=cv2.contourArea,reverse=True)
  biggest_contour=np.array([])
  max_area=0
  for i in contours:
    area=cv2.contourArea(i)
    if area>4000:
      perimeter=cv2.arcLength(i,True)
      app=cv2.approxPolyDP(i,0.02*perimeter,True)
      if area>max_area and len(app)==4:
        biggest_contour=app
        max_area=area
  return biggest_contour,max_area

In [216]:
biggest_contour,max_area=find_biggest_contour(contours)
biggest_contour

array([[[ 49,  23]],

       [[ 26, 778]],

       [[619, 780]],

       [[599,  33]]], dtype=int32)

In [217]:
cv2.drawContours(image_f_contours,biggest_contour,-1,(0,255,0),10)
show_image(image_f_contours)

In [218]:
rect=np.zeros((4,2),dtype='float32')
biggest_contour=np.squeeze(biggest_contour,axis=1)
if biggest_contour.shape[0] == 4:
    s = np.sum(biggest_contour,axis=1)
    rect[1] = biggest_contour[np.argmin(s)]
    rect[3] = biggest_contour[np.argmax(s)]
    diff = np.diff(biggest_contour, axis=1)
    rect[0] = biggest_contour[np.argmin(diff)]
    rect[2] = biggest_contour[np.argmax(diff)]
    rect.tolist()

In [219]:
warped_image=image.copy()
if biggest_contour.size!=0:
  final_points=np.float32([[img_width,0],[0,0],[0 ,img_height],[img_width,img_height]])
  perspective=cv2.getPerspectiveTransform(rect.astype(np.float32),final_points)
  warped_image=cv2.warpPerspective(image,perspective,(img_width,img_height))


In [220]:
img_width

636

In [221]:
show_image(warped_image)

In [222]:
b, g, r = cv2.split(warped_image)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
blue = clahe.apply(b)
green = clahe.apply(g)
red = clahe.apply(r)
equ = cv2.merge((blue, green, red))
show_image(equ)
